In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import intake

import pandas as pd

import dask
from dask.distributed import Client

from joblib import load

from metsim.methods import mtclim
from fluxnet_etl import get_fluxnet

In [3]:
!mkdir -p metsim_data

In [4]:
x_data_computed, y_data_computed, meta_df = get_fluxnet(from_cache=True)

In [23]:
from metsim.metsim import MetSim, wrap_run_cell
import metsim.constants as cnst

param_keys = ['sw_prec_thresh', 'lw_cloud', 'lw_type', 'tday_coef',
              'tdew_tol', 'tmax_daylength_fraction', 'rain_scalar',
              'lapse_rate', 'utc_offset', 'prec_type', 'calendar']
params = {key: MetSim.params[key] for key in param_keys}
params['time_step'] = 60
# params['utc_offset'] = False
params

{'sw_prec_thresh': 0.0,
 'lw_cloud': 'cloud_deardorff',
 'lw_type': 'prata',
 'tday_coef': 0.45,
 'tdew_tol': 1e-06,
 'tmax_daylength_fraction': 0.67,
 'rain_scalar': 0.75,
 'lapse_rate': 0.0065,
 'utc_offset': False,
 'prec_type': 'uniform',
 'calendar': 'standard',
 'time_step': 60}

In [6]:
client = Client(n_workers=20)
client

Client Scheduler: tcp://127.0.0.1:43523 Dashboard: proxy/8787/status,Cluster Workers: 20 Cores: 20 Memory: 32.21 GB


In [32]:
@dask.delayed
def run_metsim(df, site_meta):

    df['dtr'] = df['t_max'] - df['t_min']
    df['smoothed_dtr'] = df['dtr'].rolling(30).mean()
    df['seasonal_prec'] = cnst.DAYS_PER_YEAR * df['P'].rolling(90).mean()
    df['lat'] = site_meta['lat']
    df['lon'] = site_meta['lon']
    df['elev'] = site_meta['elev']
    df.index.name = 'time'

    ds = df.rename(columns={'P': 'prec'}).to_xarray().isel(time=slice(89, None))

    state = df.rename(columns={'P': 'prec'}).to_xarray().isel(time=slice(None, 90))
    disagg = True
#     out_times = ds.indexes['time']
    out_times = pd.date_range(ds.indexes['time'][0], ds.indexes['time'][-1]+pd.Timedelta('23H'), freq='1H')
    print(out_times)
    return wrap_run_cell(mtclim.run, params, ds, state, disagg, out_times)[0]


def to_daily(df):
    out = df.resample('1D').mean()
    out['prec'] = df.resample('1D').sum()
    return out


@dask.delayed
def write_metsim(df, name):
    df.to_csv(f'./metsim_data/metsim_{name}_HH.csv')
    df.pipe(to_daily).to_csv(f'./metsim_data/metsim_{name}_DD.csv')
    return name

In [33]:
tasks = []
for df, (name, site_meta) in zip(x_data_computed, meta_df.iterrows()):
    print(name)
    out = run_metsim(df, site_meta)
    tasks.append(write_metsim(out, name))

AR-SLu
AR-Vir
AT-Neu
AU-Ade
AU-ASM
AU-Cpr
AU-Cum
AU-DaP
AU-DaS
AU-Dry
AU-Emr
AU-Fog
AU-Gin
AU-GWW
AU-How
AU-Lox
AU-RDF
AU-Rig
AU-Rob
AU-Stp
AU-TTE
AU-Tum
AU-Wac
AU-Whr
AU-Wom
AU-Ync
BE-Bra
BE-Lon
BE-Vie
BR-Sa1
BR-Sa3
CA-Gro
CA-NS1
CA-NS2
CA-NS3
CA-NS4
CA-NS5
CA-NS6
CA-NS7
CA-Oas
CA-Obs
CA-Qfo
CA-SF1
CA-SF2
CA-SF3
CA-TP1
CA-TP2
CA-TP3
CA-TP4
CA-TPD
CG-Tch
CH-Cha
CH-Dav
CH-Fru
CH-Lae
CH-Oe1
CH-Oe2
CN-Cha
CN-Cng
CN-Dan
CN-Din
CN-Du2
CN-Du3
CN-Ha2
CN-HaM
CN-Qia
CN-Sw2
CZ-BK1
CZ-BK2
CZ-wet
DE-Akm
DE-Geb
DE-Gri
DE-Hai
DE-Kli
DE-Lkb
DE-Lnf
DE-Obe
DE-Seh
DE-SfN
DE-Spw
DE-Tha
DE-Zrk
DK-Eng
DK-Fou
DK-NuF
DK-Sor
DK-ZaF
DK-ZaH
ES-Amo
ES-LgS
ES-LJu
ES-Ln2
FI-Hyy
FI-Jok
FI-Let
FI-Lom
FI-Sod
FR-Fon
FR-Gri
FR-LBr
FR-Pue
GF-Guy
GH-Ank
IT-BCi
IT-CA1
IT-CA2
IT-CA3
IT-Col
IT-Cp2
IT-Cpz
IT-Isp
IT-La2
IT-Lav
IT-MBo
IT-Noe
IT-PT1
IT-Ren
IT-Ro1
IT-Ro2
IT-SR2
IT-SRo
IT-Tor
JP-MBF
JP-SMF
NL-Hor
NL-Loo
NO-Adv
NO-Blv
PA-SPn
PA-SPs
RU-Che
RU-Cok
RU-Fyo
RU-Ha1
RU-Sam
RU-SkP
RU-Tks
RU-Vrk
SD-Dem
SE-St1
SN-Dhr
US-AR1

In [34]:
dask.persist(tasks)

([Delayed('write_metsim-65a2b29e-3ccf-45bc-9da7-2811f9a0f29d'),
  Delayed('write_metsim-d70d6c78-71a3-4d05-b2b4-c64871af86bc'),
  Delayed('write_metsim-4a5f1210-9a43-4f1f-9634-34577c1e614c'),
  Delayed('write_metsim-efad5c86-ffbb-4cf1-ab43-9da93ea32bce'),
  Delayed('write_metsim-ecc18cc5-9df0-4581-82f1-d47d64809b98'),
  Delayed('write_metsim-220333be-4f49-4640-802e-9f0077e61577'),
  Delayed('write_metsim-c462effa-7fa9-49f6-a384-02907c20f762'),
  Delayed('write_metsim-d7448ca6-f96f-41f9-9d2c-8d133e598967'),
  Delayed('write_metsim-f2d9fd1e-33b5-465c-b505-7af0b35a575b'),
  Delayed('write_metsim-f88c3615-3466-4052-b416-7bd22080c029'),
  Delayed('write_metsim-0619b5ca-2889-4116-a9f0-d77ac34e05f4'),
  Delayed('write_metsim-89aa9eb3-2d04-442d-8b4e-445ed631b5f7'),
  Delayed('write_metsim-99f9bb29-2946-46ba-9e9f-d66c7a051976'),
  Delayed('write_metsim-b6c74944-218c-4747-9ce8-5de43292d23f'),
  Delayed('write_metsim-dd5182c6-a8a7-4b93-8724-8b98b1a9a96e'),
  Delayed('write_metsim-9bc31c68-e2f7-49